# Assignment 

**Import relevant packages**

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydst
from pandas_datareader import wb
import geoplotlib

In [27]:
Dst = pydst.Dst(lang='da')

Overview of possible datasets

In [28]:
Dst.get_subjects()

,active,desc,hasSubjects,id
0,True,Befolkning og valg,True,02
1,True,Levevilkår,True,05
2,True,Uddannelse og viden,True,03
3,True,Kultur og kirke,True,18
4,True,"Arbejde, indkomst og formue",True,04
5,True,Priser og forbrug,True,06
6,True,Nationalregnskab og offentlige finanser,True,14
7,True,Penge og kapitalmarked,True,16
8,True,Udenrigsøkonomi,True,13
9,True,Erhvervslivet på tværs,True,07


We choose the category 'Levevilkår' (living conditions)

In [30]:
Dst.get_tables(subjects=['05']).head()

,active,firstPeriod,id,latestPeriod,text,unit,updated,variables
0,True,2008K1,FOLK1A,2019K1,Folketal den 1. i kvartalet,antal,2019-02-11 08:00:00,"[område, køn, alder, civilstand, tid]"
1,True,2008K1,FOLK1B,2019K1,Folketal den 1. i kvartalet,antal,2019-02-11 08:00:00,"[område, køn, alder, statsborgerskab, tid]"
2,True,2008K1,FOLK1C,2019K1,Folketal den 1. i kvartalet,antal,2019-02-11 08:00:00,"[område, køn, alder, herkomst, oprindelsesland..."
3,True,2008K1,FOLK1D,2019K1,Folketal den 1. i kvartalet,antal,2019-02-11 08:00:00,"[område, køn, alder, statsborgerskab, tid]"
4,True,2008K1,FOLK1E,2019K1,Folketal den 1. i kvartalet,antal,2019-02-11 08:00:00,"[område, køn, alder, herkomst, tid]"


Within the category 'living conditions' we choose 'straf11' which contains data on reported criminal offences by region and type since 1st quarter 2007 to 4th quarter 2018. 

In [31]:
Dst.get_data(table_id = 'straf11')

,OMRÅDE,OVERTRÆD,TID,INDHOLD
0,Hele landet,Overtrædelsens art i alt,2007K1,121167


From the table above we note that the dataset contains 121.167 observations on reported criminal offences from the whole country.

In [ ]:
crime_api = Dst.get_data(table_id = 'straf11',
                         variables={'OMRÅDE':['*'], 
                                    'OVERTRÆDELSENS ART':['*'], 
                                    'KVARTAL':['*']})


In [ ]:
type(crime_api)

In [7]:
Entry_wb = wb.download(indicator='IC.BUS.NREG', country=['SE','DK','NO'], start=2016, end=2016)

In [16]:
Entry_wb = Entry_wb.rename(columns = {'IC.BUS.NREG':'Number of new entrants'})
#Entry_wb = Entry_wb.rename(columns = {'IC.BUS.NREG':'Number of new entrants'})
Entry_wb = Entry_wb.reset_index()
Entry_wb.head(10)

,level_0,index,country,year,Number of new entrants
0,0,0,Denmark,2016,36384
1,1,1,Norway,2016,27920
2,2,2,Sweden,2016,49960


In [18]:
geoplotlib.dot(Entry_wb.loc[:,['country','Number of new entrants']])
geoplotlib.show()


Traceback (most recent call last):
  File "C:\Users\Lykke Sterll\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3078, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 140, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 162, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1492, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1500, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'lon'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Lykke Sterll\Anaconda3\lib\site-packages\geoplotlib\__init__.py", line 32, in _runapp
    app.start()
  File "C:\Users\Lykke Sterll\Anaconda3\lib\site-packages\geoplotlib\core.py", line 364, in start
    self.proj.fit(BoundingBox.from_bboxes([l.bbox() for l in self.geoplotlib_confi